In [3]:
from hydra import compose, initialize

from dataset import get_transforms, load_datalist

with initialize(version_base=None, config_path="config", job_name="test_app"):
    cfg = compose(config_name="root",
                  overrides=['data=task8', 'data.root=D:/NTUH', 'debug=false', 'data.normalize_values=[0,0]',
                             'data.clip_values=[0,0]', 'data.train.images=images/*.nii.gz',
                             'data.train.labels=labels/*.nrrd', 'data.test.images=images/*.nii.gz',
                             'data.train.label_extension=nrrd'])

datalist = load_datalist(cfg)

transform = get_transforms('test', cfg, no_aug=True)

In [15]:
from pathlib import Path
import os
import shutil

Path(r'D:\NTUH\labels_tumor').mkdir(exist_ok=True, parents=True)

for image_path in Path(r'D:\NTUH\images').glob('*.nii.gz'):
    if os.path.exists(str(image_path).replace(r'NTUH\images', r'Task08_HepaticVessel\labelsTr')):
        shutil.copy(str(image_path).replace(r'NTUH\images', r'Task08_HepaticVessel\labelsTr'),
                    str(image_path).replace(r'NTUH\images', r'NTUH\labels_tumor'))

In [71]:
import nibabel as nb
import numpy as np

Path(r'D:\NTUH\combined_labels').mkdir(exist_ok=True, parents=True)

for tumor_label_path in Path(r'D:\NTUH\labels_tumor').glob('*.nii.gz'):
    label_path = str(tumor_label_path).replace(r'NTUH\labels_tumor', r'NTUH\labels')
    
    lab_img = nb.load(label_path)
    tumor_img = nb.load(tumor_label_path)
    
    lab_data = np.asarray(lab_img.dataobj)
    tumor_lab_data = np.asarray(tumor_img.dataobj)
    
    lab_data[tumor_lab_data == 2] = 4
    
    new_file = str(tumor_label_path).replace(r'NTUH\labels_tumor', r'NTUH\combined_labels')
    new_img = nb.Nifti1Image(lab_data, lab_img.affine, lab_img.header)
    nb.save(new_img, new_file)